# Analysis of solved instances (Part 1)
Comparing:
1. Zonotope Propagation with LP branching check and LP fallback for equivalence check (`091c775`)
2. Same as above but with CEGAR overapproximation using `np.argmax(np.sum(np.abs(mat[:, self.input_size:]), axis=0))` as refinement strategy; so far only epsilon equiv (`c5c8f02`)

In [1]:
import parse
import matplotlib.pyplot as plt
import numpy as np

In [2]:
benchmarks = [
    "../results/ACASXU_run2a_1_1_batch_2000-trunc-1010-0.05/",
    "../results/ACASXU_run2a_1_2_batch_2000-trunc-1010-0.05/",
    "../results/HAR-trunc-1000.1-0.25/",
    "../results/HAR-trunc-1000-0.25/",
    "../results/mnist_relu_3_100-trunc-400.1-0.25/",
    "../results/mnist_relu_3_100-trunc-400-0.25/",
    "../results/mnist8x8_student_12_12_12_10-9000.5-top/",
    "../results/mnist8x8_student_12_12_12_10-9100.5-top/",
    "../results/mnist8x8_student_24_12_10-9000.5-top/",
    "../results/mnist8x8_student_24_12_10-9100.5-top/",
    "../results/mnist8x8_student_36_10-9000.5-top/",
    "../results/mnist8x8_student_36_10-9100.5-top/"
]

In [3]:
pure_zono = {}
cegar_zono = {}
for b in benchmarks:
    try:
        pure_zono[b]=parse.BenchmarkRun(
            b+"1/nnequiv-091c775/stdout.log",
            b+"1/nnequiv-091c775/stderr.log",
            out_handlers=[parse.EquivLine],
            err_handlers=[parse.RunLim]
            )
    except FileNotFoundError:
        print(f"Could not find {b}")
        pure_zono[b]=None
    try:
        cegar_zono[b]=parse.BenchmarkRun(
            b+"1/nnequiv-c5c8f02/stdout.log",
            b+"1/nnequiv-c5c8f02/stderr.log",
            out_handlers=[parse.EquivLine],
            err_handlers=[parse.RunLim]
            )
    except FileNotFoundError:
        print(f"Could not find {b}")
        cegar_zono[b]=None

Could not find ../results/mnist_relu_3_100-trunc-400-0.25/
Could not find ../results/mnist8x8_student_12_12_12_10-9000.5-top/
Could not find ../results/mnist8x8_student_12_12_12_10-9100.5-top/
Could not find ../results/mnist8x8_student_24_12_10-9000.5-top/
Could not find ../results/mnist8x8_student_24_12_10-9100.5-top/
Could not find ../results/mnist8x8_student_36_10-9000.5-top/
Could not find ../results/mnist8x8_student_36_10-9100.5-top/


In [4]:
import analyse

In [5]:
comparison = analyse.RunlimComparator(pure_zono,cegar_zono,benchmarks)
    
comparison.render_table()

,Name,Status (1),Status (2),Real (1),Real (2),Time (1),Time (2),Mem (1),Mem (2)
0,../results/ACASXU_run2a_1_1_batch_2000-trunc-1010-0.05/,ok,ok,422.26,1185.97,421.64,1217.65,223.2,204.2
1,../results/ACASXU_run2a_1_2_batch_2000-trunc-1010-0.05/,ok,ok,661.97,1349.35,661.14,1391.36,292.8,222.7
2,../results/HAR-trunc-1000.1-0.25/,ok,ok,84.2,2.69,147.88,2.46,160.9,127.8
3,../results/HAR-trunc-1000-0.25/,out,out,10800.2,10800.1,17692,11797.4,416.5,386.1
4,../results/mnist_relu_3_100-trunc-400.1-0.25/,ok,ok,22.44,2.28,41.16,2.03,156.7,125.5
5,../results/mnist_relu_3_100-trunc-400-0.25/,None,out,nan,10800.1,nan,11507.6,nan,286.1
6,../results/mnist8x8_student_12_12_12_10-9000.5-top/,ok,None,75.52,nan,75.21,nan,116.7,nan
7,../results/mnist8x8_student_12_12_12_10-9100.5-top/,ok,None,242.38,nan,242.06,nan,119.9,nan
8,../results/mnist8x8_student_24_12_10-9000.5-top/,ok,None,41.22,nan,40.81,nan,115.5,nan
9,../results/mnist8x8_student_24_12_10-9100.5-top/,ok,None,119.91,nan,119.34,nan,117.1,nan


# Next comparison...
1. Zonotope Propagation with LP branching check and LP fallback for equivalence check (`091c775`)
2. Zonotope Propagation with LP branching check and LP fallback.  
Additionally, we initially overapproximated all ReLU nodes which have a *phase transition* and refine if necessary with a CEGAR like approach. (`db06768`)
    - Strategy for epsilon: Max column sum in generator matrix (same as above)
    - Strategy for top k: First ReLU is done exact instead of using overapprox.

**Additionally** we fixed an error of thought in the approach above:
For the ReLU nodes above we only ran the LP for the variables of the input space. Here we added all variables.
This, for sure, is sound. The other one needs to be further investigated.
**Drawback:** growth of LP dimensionality

In [8]:
cegar2_zono = {}
for b in benchmarks:
    try:
        cegar2_zono[b]=parse.BenchmarkRun(
            b+"1/nnequiv-db06768/stdout.log",
            b+"1/nnequiv-db06768/stderr.log",
            out_handlers=[parse.EquivLine],
            err_handlers=[parse.RunLim]
            )
    except FileNotFoundError:
        print(f"Could not find {b}")
        cegar2_zono[b]=None

Could not find ../results/HAR-trunc-1000-0.25/
Could not find ../results/mnist_relu_3_100-trunc-400-0.25/


In [9]:
comparison = analyse.RunlimComparator(pure_zono,cegar2_zono,benchmarks)
    
comparison.render_table()

,Name,Status (1),Status (2),Real (1),Real (2),Time (1),Time (2),Mem (1),Mem (2)
0,../results/ACASXU_run2a_1_1_batch_2000-trunc-1010-0.05/,ok,out,422.26,10800.1,421.64,10793.9,223.2,311.4
1,../results/ACASXU_run2a_1_2_batch_2000-trunc-1010-0.05/,ok,out,661.97,10800.1,661.14,10802.1,292.8,362.3
2,../results/HAR-trunc-1000.1-0.25/,ok,ok,84.2,2.53,147.88,2.66,160.9,126.7
3,../results/HAR-trunc-1000-0.25/,out,None,10800.2,nan,17692,nan,416.5,nan
4,../results/mnist_relu_3_100-trunc-400.1-0.25/,ok,ok,22.44,2.19,41.16,2.03,156.7,126
5,../results/mnist_relu_3_100-trunc-400-0.25/,None,None,nan,nan,nan,nan,nan,nan
6,../results/mnist8x8_student_12_12_12_10-9000.5-top/,ok,ok,75.52,12.35,75.21,12.21,116.7,114.3
7,../results/mnist8x8_student_12_12_12_10-9100.5-top/,ok,ok,242.38,1.41,242.06,1.31,119.9,113.7
8,../results/mnist8x8_student_24_12_10-9000.5-top/,ok,ok,41.22,11.46,40.81,11.21,115.5,114.3
9,../results/mnist8x8_student_24_12_10-9100.5-top/,ok,ok,119.91,1.74,119.34,1.42,117.1,113.7


## What we see...
- Our approach is extremely efficient for proving the top k MNIST cases
    - Investigate if same goes for other dimensionalities (mnist8x8 has 64 input dimensions)
    - Investigate if same goes for other networks
- Our approach is very efficient for small subsets (see HAR 1000.1 and mnist_relu_3_100 400.1) where overapproximation is doing it on its own or very few refinements are necessary
- Our approach is currently very bad for epsilon equivalence. This seems to be the case due to LP time (higher dimensionality!) and layer processing (higher dimensionality!)

## What we need to do...
- Run experiment with EGO instead of CEGAR (in the pipeline already)
- Investigate more networks with top k equivalence
- Explore methods which avoid the LP growth
- Save some things that are recomputed (paths through the network could be cached?)
- Better refinement heuristics (especially for the epsilon case)  
**Just a thought**: Can we compute the derivative of the epsilon error towards the relu overapprox error?
If so overapprox error * its error derivative might be a viable solution to our issue...